In [ ]:
import os
import re
import pandas as pd

In [ ]:
common_voice_dir = 'path/to/your/data/CV/cv-corpus-20.0-2024-12-06-nl/cv-corpus-20.0-2024-12-06/nl' ###
validated_path = os.path.join(common_voice_dir, "validated.tsv")
durations_path = os.path.join(common_voice_dir, "clip_durations.tsv")

In [ ]:
df_transcripts = pd.read_csv(validated_path, sep='\t')
df_durations = pd.read_csv(durations_path, sep='\t')

df = df_transcripts.merge(df_durations, left_on='path', right_on='clip')

stm_lines = []

#convert ms to seconds
df['start_time'] = 0.00
df['end_time'] = df['duration[ms]'] / 1000.0

def make_speaker_tag(row):
    age = row['age'] if pd.notna(row['age']) else ""
    gender = row['gender'] if pd.notna(row['gender']) else ""
    accent = row['accents'] if pd.notna(row['accents']) else ""
    combined = "_".join(part.replace(" ", "-") for part in [age, gender, accent] if part)
    return f"<o,f0,{combined or 'unknown'}>"

stm_lines = []
for _, row in df.iterrows():
    filename = row['path'].replace(".mp3", "").replace(".wav", "")
    speaker = row['client_id']
    transcript = row['sentence'].strip().replace("(", "-LRB-").replace(")", "-RRB-")
    meta_tag = make_speaker_tag(row)
    
    stm_line = f"{filename} 1 {speaker} {row['start_time']:.2f} {row['end_time']:.2f} {meta_tag} {transcript}"
    stm_lines.append(stm_line)

with open("CV_reference.stm", "w", encoding="utf-8") as f:
    f.write("\n".join(stm_lines))

C:\Users\Topicus\AppData\Local\Temp\ipykernel_7652\2684508545.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transcripts = pd.read_csv(validated_path, sep='\t')


In [ ]:
#fixing the names in the output file

reference_file = "path/to/your/data/CV_reference.stm" ###
with open(reference_file, "r", encoding="utf-8") as f:
    lines = f.readlines()
    updated_lines = []
    for line in lines:
        if line.startswith('common_voice_nl'):
            parts = line.split(' ', 1)
            parts[0] = parts[0].replace('common_voice_nl', 'CV')
            updated_line = ' '.join(parts)
        else:
            updated_line = line
        updated_lines.append(updated_line)

    with open(reference_file, 'w', encoding='utf-8') as f:
        f.writelines(updated_lines)